In [ ]:
import os
import feast

actual_version = feast.__version__
assert actual_version == os.environ.get("FEAST_VERSION"), (
    f"❌ Feast version mismatch. Expected: {os.environ.get('FEAST_VERSION')}, Found: {actual_version}"
)
print(f"✅ Found Expected Feast version: {actual_version} in workbench")

In [ ]:
# --- Configuration Variables ---
import os 

# Fetch token and server directly from oc CLI
import subprocess

def oc(cmd):
    return subprocess.check_output(cmd, shell=True).decode("utf-8").strip()

token = oc("oc whoami -t")
server = oc("oc whoami --show-server")
namespace = os.environ.get("NAMESPACE")


In [ ]:
!oc login --token=$token --server=$server

In [ ]:
# Add user permission to namespace
!oc adm policy add-role-to-user admin $(oc whoami) -n $namespace

In [ ]:
import os

namespace = os.environ.get("NAMESPACE")  # read namespace from env
if not namespace:
    raise ValueError("NAMESPACE environment variable is not set")

yaml_content = os.popen(
    f"oc get configmap feast-credit-scoring-client -n {namespace} "
    "-o jsonpath='{.data.feature_store\\.yaml}' | sed 's/\\\\n/\\n/g'"
).read()

# Save the configmap data into an environment variable (if needed)
os.environ["CONFIGMAP_DATA"] = yaml_content

In [ ]:
from feast import FeatureStore
fs_credit_scoring_local = FeatureStore(fs_yaml_file='/opt/app-root/src/feast-config/credit_scoring_local')

In [ ]:
project_name = "credit_scoring_local"
project = fs_credit_scoring_local.get_project(project_name)

# 1. Assert object returned
assert project is not None, f"❌ get_project('{project_name}') returned None"

# 2. Extract project name (works for dict or Feast object)
if isinstance(project, dict):
    returned_name = project.get("spec", {}).get("name")
else:
    # Feast Project object
    returned_name = getattr(project, "name", None)
    if not returned_name and hasattr(project, "spec") and hasattr(project.spec, "name"):
        returned_name = project.spec.name

# 3. Assert that name exists
assert returned_name, f"❌ Returned project does not contain a valid name: {project}"

print("• Project Name Returned:", returned_name)

# 4. Assert the name matches expected
assert returned_name == project_name, (
    f"❌ Expected project '{project_name}', but got '{returned_name}'"
)

print(f"\n✓ get_project('{project_name}') validation passed!")


In [ ]:
feast_list_functions = [
    "list_projects",
    "list_entities",
    "list_feature_views",
    "list_all_feature_views",
    "list_batch_feature_views",
    "list_on_demand_feature_views",
]

# validates feast list methods returns data and method type
def validate_list_method(fs_obj, method_name):
    assert hasattr(fs_obj, method_name), f"Method not found: {method_name}"

    method = getattr(fs_obj, method_name)
    result = method()

    assert isinstance(result, list), (
        f"{method_name}() must return a list, got {type(result)}"
    )
    assert len(result) > 0, (
        f"{method_name}() returned an empty list — expected data"
    )

    print(f"✓ {method_name}() returned {len(result)} items")

for m in feast_list_functions:
    validate_list_method(fs_credit_scoring_local, m)


In [ ]:
feast_list_functions = [
    "list_feature_services",
    # "list_permissions",
    "list_saved_datasets",
]

# validates feast methods exists and type is valid
def validate_list_func(fs_obj, method_name):
    assert hasattr(fs_obj, method_name), f"Method not found: {method_name}"

    method = getattr(fs_obj, method_name)

    result = method()

    assert isinstance(result, list), (
        f"{method_name}() must return a list, got {type(result)}"
    )

for m in feast_list_functions:
    validate_list_func(fs_credit_scoring_local, m)

In [ ]:
# validate_list_data_sources for with and without permissions 

import os
from feast.errors import FeastPermissionError

def validate_list_data_sources(fs_obj):
    """
    Validates list_data_sources() with special handling for Kubernetes auth mode.
    If CONFIGMAP_DATA indicates auth=kubernetes, expect FeastPermissionError.
    Otherwise validate output type normally.
    """
    auth_mode = os.getenv("CONFIGMAP_DATA")

    # Case 1: Kubernetes auth → expect permission error
    if "kubernetes" in auth_mode.lower():
        try:
            fs_obj.list_data_sources()
            raise AssertionError(
                "Expected FeastPermissionError due to Kubernetes auth, but the call succeeded."
            )
        except FeastPermissionError as e:
            # Correct, this is expected
            return
        except Exception as e:
            raise AssertionError(
                f"Expected FeastPermissionError, but got different exception: {type(e)} - {e}"
            )

    # Case 2: Non-Kubernetes auth → normal path
    assert hasattr(fs_obj, "list_data_sources"), "Method not found: list_data_sources"
    result = fs_obj.list_data_sources()
    assert isinstance(result, list), (
        f"list_data_sources() must return a list, got {type(result)}"
    )


In [ ]:

entity = fs_credit_scoring_local.get_entity("dob_ssn")

assert entity is not None, "❌ Entity 'dob_ssn' not found!"
assert entity.name == "dob_ssn", f"❌ Entity name mismatch: {entity.name}"

print("✓ Entity validation successful!\n", entity.name)

In [ ]:

fv = fs_credit_scoring_local.get_feature_view("credit_history")

assert fv is not None, "❌ FeatureView 'credit_history' not found!"
assert fv.name == "credit_history", f"❌ Name mismatch: {fv.name}"

print("• FeatureView : validation successful!", fv.name)

In [ ]:
import os
from feast.errors import FeastPermissionError

def validate_get_data_source(fs_obj, name: str):
    auth_mode = os.getenv("CONFIGMAP_DATA", "")

    print("📌 CONFIGMAP_DATA:", auth_mode)

    # If Kubernetes auth is enabled → expect permission error
    if "auth" in "kubernetes" in auth_mode.lower():
        print(f"🔒 Kubernetes auth detected, expecting permission error for get_data_source('{name}')")

        try:
            fs_obj.get_data_source(name)
            raise AssertionError(
                f"❌ Expected FeastPermissionError when accessing data source '{name}', but call succeeded"
            )

        except FeastPermissionError as e:
            print(f"✅ Correctly blocked with FeastPermissionError: {e}")
            return

        except Exception as e:
            raise AssertionError(
                f"❌ Expected FeastPermissionError but got {type(e)}: {e}"
            )

    # Otherwise → normal validation
    print(f"🔍 Fetching data source '{name}'...")

    ds = fs_obj.get_data_source(name)

    print("\n📌 Data Source Object:")
    print(ds)

    assert ds.name == name, (
        f"❌ Expected name '{name}', got '{ds.name}'"
    )

    print(f"✅ Data source '{name}' exists and is correctly configured.")


In [ ]:
feast_features = [
    "zipcode_features:city",
    "zipcode_features:state",
]

entity_rows = [{
    "zipcode": 1463,
    "dob_ssn": "19530219_5179"
}]

response = fs_credit_scoring_local.get_online_features(
    features=feast_features,
    entity_rows=entity_rows,
).to_dict()

print("Actual response:", response)

expected = {
    'zipcode': [1463],
    'dob_ssn': ['19530219_5179'],
    'city': ['PEPPERELL'],
    'state': ['MA'],
}

assert response == expected

In [ ]:
import pandas as pd

# Input entity dataframe
entity_df = pd.DataFrame({
    "dob_ssn": ["19530219_5179"],
    "zipcode": [1463],
    "event_timestamp": [pd.Timestamp("2020-04-26 18:01:04")]
})

feast_features = [
    "zipcode_features:city",
    "zipcode_features:state",
    "credit_history:credit_card_due",
    "credit_history:mortgage_due",
]

# Retrieve historical features
historical_df = fs_credit_scoring_local.get_historical_features(
    entity_df=entity_df,
    features=feast_features,
).to_df()

print("Historical DF:\n", historical_df)

# Validate dataframe is not empty
assert not historical_df.empty, " Historical features dataframe is empty!"

# 2. Validate required columns exist
expected_cols = {
    "dob_ssn", "zipcode", "event_timestamp",
    "city", "state",
    "credit_card_due", "mortgage_due"
}

missing_cols = expected_cols - set(historical_df.columns)
assert not missing_cols, f" Missing columns in result: {missing_cols}"

# 3. Validate city/state are non-null (critical features)
assert pd.notna(historical_df.loc[0, "city"]), " 'city' value is null!"
assert pd.notna(historical_df.loc[0, "state"]), " 'state' value is null!"

# 4. Validate entity matches input
assert historical_df.loc[0, "zipcode"] == 1463, " zipcode mismatch!"
assert historical_df.loc[0, "dob_ssn"] == "19530219_5179", "❌ dob_ssn mismatch!"

print("✅ All validations passed successfully!")
